In [11]:
# Challenge 1

### Initial datalog code
# (declare-rel inv (Int Int ))
# (declare-var x0 Int)
# (declare-var x1 Int)
# (declare-var y0 Int)
# (declare-var y1 Int)

# (declare-rel fail ())

# (rule (=> (and (= x0 0) (= y0 5000))
#     (inv x0 y0)))

# (rule (=> (and
#         (inv x0 y0)
#         (= x1 (+ x0 1))
#         (= y1 (ite (>= x0 5000) (+ y0 1) y0)))
#     (inv x1 y1)))

# (rule (=> (and (inv x0 y0) (= x0 10000)
#     (not (= y0 x0))) fail))

# (query fail)
### Solve
from z3 import *
from spacer_tutorial import *
def mk_cfa1():
    A = CFA('prog')
    x, x_out = A.add_var(z3.IntSort(), name='x')
    y, y_out = A.add_var(z3.IntSort(), name='y')
    A.set_entry_node('init0')
    A.set_exit_node('fail')

    N = 5000
    M = N*2
    A.add_edge('init0', 'init', z3.And(x == 0, y == N))
    A.add_edge('init', 'inv', z3.And(x_out == x, y_out == y, x <= y))
    A.add_edge('inv', 'inv', z3.And(x_out == x + 1, y_out == z3.If(x >= N, y + 1, y)))
    A.add_assert('inv', z3.Implies(x == M, x == y))

    return A
cfa1 = mk_cfa1()
cfa1.to_dot()
#### In TR fn I provided additional condition x < y. It makes things work
def mk_challenge_1_orig(N = 0):
    T = Ts('Ts0')
    M = N * 2
    x, x_out = T.add_var(z3.IntSort(), name='x')
    y, y_out = T.add_var(z3.IntSort(), name='y')
    T.Init = And(x == 0, y == N)
    T.Tr = And(x_out == x + 1, y_out == If(x >= N, y + 1, y))
    T.Bad = And(x == M, Not(y == x))
    return T
def mk_challenge_1(N = 5000):
    T = Ts('Ts0')
    x, x_out = T.add_var(z3.IntSort(), name='x')
    y, y_out = T.add_var(z3.IntSort(), name='y')
    z, z_out = T.add_var(z3.IntSort(), name='z')
    n, n_out = T.add_var(z3.IntSort(), name='n')

    T.Init = And(x == 0, y == z, z >= 0, z <= N, n == 2*N)
    T.Tr = And(z_out == z, 
               x_out == x + 1, 
               n_out == n, 
               y_out == If(x >= z, y + 1, y))
    T.Bad = And(x == n, Not(y == x))
    return T
ts_challenge_1 = mk_challenge_1()
HtmlStr(ts_challenge_1)
def vc_challenge_1(T):
    Inv = z3.Function('Inv', *(T.sig() + [z3.BoolSort()]))

    InvPre = Inv(*T.pre_vars())
    InvPost = Inv(*T.post_vars())

    all_vars = T.all()
    vc_init = z3.ForAll(all_vars, z3.Implies(T.Init, InvPre))
    vc_ind = z3.ForAll(all_vars, z3.Implies(z3.And(InvPre, T.Tr), InvPost))
    vc_bad = z3.ForAll(all_vars, z3.Implies(z3.And(InvPre, T.Bad), z3.BoolVal(False)))
    return [vc_init, vc_ind, vc_bad], InvPre

vc_chall_1, inv_challenge_1 = vc_challenge_1(ts_challenge_1)
res_challenge_1, answer_challenge_1 = solve_horn(vc_chall_1, max_unfold=40)
res_challenge_1

#SpacerProof(answer_challenge_1)
### Answer
answ_ch_1 = answer_challenge_1.eval(inv_challenge_1)
answ_ch_1
push_not(answ_ch_1)
#def ch_1_program(N):
    # vc_ch_1, inv_ch_1 = vc_challenge_1(mk_challenge_1(N))
    # res_ch_1, answer_ch_1 = solve_horn(vc_ch_1, max_unfold=40)
    # a_ch_1 = answer_ch_1.eval(inv_ch_1)
    # return push_not(a_ch_1)

#ch_1_program(1)
#for x in [*range(20)]:
#   chc = ch_1_program(x)
#   print(f"N = {x}: {chc}")
  
  

And(Implies(x < y, y <= z), 1 <= n + -1*z, y >= z, x <= y)